In [20]:
# External libraries
import pandas as pd

In [21]:
path = 'https://raw.githubusercontent.com/compliancemarketplace/pythonistas/main/challenges/challenge_3/data/order.parquet'

In [22]:
original_data = pd.read_parquet(path) # Keep original data safe

In [23]:
 # Coping from original data, to work  with
data = original_data.copy()
data.head(n=3)

,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal


In [24]:
def normalize_columns(columns: list) -> list:
    """
    Normalize dataframe columns

    Args:
        columns: A list with column names
    
    Yields:
        Normalized column names
    """
    for column in columns:
        yield column.lower().replace(' ', '_')

In [26]:
# Normalizing columns
data.columns = [i for i in normalize_columns(data.columns)]
data.head()

,order_id,order_date,customername,state,city
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata


### Group By

reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

In [27]:
orders_per_day = data[['order_id', 'order_date']]
orders_per_day = orders_per_day.groupby(by='order_date').count()

In [28]:
orders_per_day.head()

,order_id
order_date,
01-01-2019,1
01-02-2019,1
01-03-2019,1
01-04-2018,2
01-05-2018,2


In [30]:
orders_per_day['order_date']

KeyError: ignored

In [31]:
orders_per_day['order_id']

order_date
01-01-2019    1
01-02-2019    1
01-03-2019    1
01-04-2018    2
01-05-2018    2
             ..
31-05-2018    1
31-07-2018    1
31-08-2018    1
31-10-2018    1
31-12-2018    1
Name: order_id, Length: 307, dtype: int64

In [38]:
orders_per_day = orders_per_day.reset_index()\
                                .rename({'order_id': 'quantity_of_orders'}, axis=1)\
                                .sort_values(by='quantity_of_orders', ascending=False)

In [39]:
orders_per_day.head()

,order_date,quantity_of_orders
240,24-11-2018,7
118,13-01-2019,6
238,24-09-2018,6
96,10-10-2018,6
289,29-10-2018,6


### Filtering

#### Using comparision operators

Get Greater than of dataframe and other, element-wise (binary operator gt).

Among flexible wrappers (eq, ne, le, lt, ge, gt) to comparison operators.

Equivalent to ==, !=, <=, <, >=, > with support to choose axis (rows or columns) and level for comparison.

reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.gt.html

In [50]:
# Equals
orders_per_day.eq(pd.Series(['24-11-2018'	, 7], index=["order_date", "quantity_of_orders"]))

,order_date,quantity_of_orders
240,True,True
118,False,False
238,False,False
96,False,False
289,False,False
...,...,...
124,False,False
125,False,False
126,False,False
127,False,False


In [51]:
# Greater than
orders_per_day.gt(pd.Series([3], index=["quantity_of_orders"]))

,order_date,quantity_of_orders
240,False,True
118,False,True
238,False,True
96,False,True
289,False,True
...,...,...
124,False,False
125,False,False
126,False,False
127,False,False


#### Using query method

Query the columns of a DataFrame with a boolean expression.

reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html

In [54]:
minimum_of_orders = 3
orders_per_day.query("quantity_of_orders >= @minimum_of_orders")

,order_date,quantity_of_orders
240,24-11-2018,7
118,13-01-2019,6
238,24-09-2018,6
96,10-10-2018,6
289,29-10-2018,6
27,03-11-2018,6
146,15-11-2018,6
254,26-04-2018,6
35,04-12-2018,6
253,26-03-2019,5


#### Using isin

Whether each element in the DataFrame is contained in values.

reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.isin.html

In [62]:
orders_per_day[orders_per_day.isin({'quantity_of_orders': range(0,4)})].

,order_date,quantity_of_orders
240,NaN,NaN
118,NaN,NaN
238,NaN,NaN
96,NaN,NaN
289,NaN,NaN
...,...,...
124,NaN,1.0
125,NaN,1.0
126,NaN,1.0
127,NaN,1.0


#### Using iloc

Purely integer-location based indexing for selection by position

reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html

In [66]:
orders_per_day.head(n=10)

,order_date,quantity_of_orders
240,24-11-2018,7
118,13-01-2019,6
238,24-09-2018,6
96,10-10-2018,6
289,29-10-2018,6
27,03-11-2018,6
146,15-11-2018,6
254,26-04-2018,6
35,04-12-2018,6
253,26-03-2019,5


In [64]:
# Index from 0 to 10
orders_per_day.iloc[0:10]

,order_date,quantity_of_orders
240,24-11-2018,7
118,13-01-2019,6
238,24-09-2018,6
96,10-10-2018,6
289,29-10-2018,6
27,03-11-2018,6
146,15-11-2018,6
254,26-04-2018,6
35,04-12-2018,6
253,26-03-2019,5


In [68]:
# Line, Col (x, y)
orders_per_day.iloc[0,0]

'24-11-2018'